In [29]:
#libraries
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine, euclidean, hamming
from sklearn.preprocessing import normalize

In [30]:
#read the data & have randome sample for reproducability
recipes = pd.read_csv('../data/full_clean_data.csv')

In [31]:
##rename the dataframe id
recipes.index.names = ['recipes_id']
recipes.head(2)

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed
recipes_id,,,,,,,,,,,,,,
0,Dessert Crepes,https://www.allrecipes.com/recipe/19037/desser...,breakfast-and-brunch,4.80,1156,"['4 eggs, lightly beaten ', ' 1\u2009⅓ cups m...",20,8,8 crepes,163.8,17.2,7.7,6.4,"['milk', 'flour', 'sugar']"
1,Chicken Parmesan,https://www.allrecipes.com/recipe/223042/chick...,world-cuisine,4.83,4245,"['4 skinless, boneless chicken breast halves ...",60,4,4 servings,470.8,24.8,24.9,42.1,"['chicken breast', 'egg', 'panko bread', 'parm..."


In [32]:
##normalize the numerical column (total+nutritional values) in new dataframe << because i dont want to normalize all other features
newrecipes = recipes[['total', 'calories', 'carbohydrates_g', 'fat_g', 'protein_g']]
recipes_norm = pd.DataFrame(normalize(newrecipes, axis=0))
recipes_norm.columns = newrecipes.columns
recipes_norm.index = newrecipes.index
# save dataframe to pickle file
recipes_norm.to_pickle('recipes_norm.pkl')

In [33]:
##convert index to column
recipes['recipes_id'] = recipes.index


In [34]:

#####creare a function for selected recipe. Guidance reference https://www.kaggle.com/code/yyzz1010/content-based-filtering-recipe-recommender?scriptVersionId=21219821&cellId=22 
def selected_recipe(recipes_id):
    recipe_df = recipes.set_index('recipes_id') # set a new index to this DataFrame
    x = "{}  {}".format(recipes_id, recipe_df.at[recipes_id, 'name']) #placeholders 
    print(x)

In [35]:
#####creare a function that recommends top 3 similar recipe using cosine. Guidance reference https://www.kaggle.com/code/yyzz1010/content-based-filtering-recipe-recommender?scriptVersionId=21219821&cellId=23
def recipe_recommender(recipes_id):
    
     allRecipes = pd.DataFrame(recipes_norm.index)
     allRecipes = allRecipes[allRecipes.recipes_id != recipes_id]
     allRecipes["distance"] = allRecipes["recipes_id"].apply(lambda x: cosine(recipes_norm.loc[recipes_id], recipes_norm.loc[x]))
     Top3NRecommendation = allRecipes.sort_values(["distance"]).head(3).sort_values(by=['distance', 'recipes_id'])
     # sort by distance then recipe id, the smaller value of recipe id will be picked. 
    
     recipe_df = recipes.set_index('recipes_id')
     recipes_id = [recipes_id]
     recipe_list = []
     for recipeid in Top3NRecommendation.recipes_id:
        recipes_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'url']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'name']))
     return recipe_list

In [36]:
selected_recipe(25)

25  Simple Roasted Butternut Squash


In [37]:
recipe_recommender(25)

['15066  https://www.allrecipes.com/recipe/236593/gluten-free-moist-choc-chip-banana-bread/',
 '15066  Gluten-Free Moist Choc Chip Banana Bread',
 '8757  https://www.allrecipes.com/recipe/45223/coffee-date-bread/',
 '8757  Coffee Date Bread',
 '14784  https://www.allrecipes.com/recipe/229967/mini-chocolate-chip-pumpkin-muffins/',
 '14784  Mini Chocolate Chip Pumpkin Muffins']

In [38]:
##create a new row to the dataframe for user input & add it again to the dataframe
user_df = {'name': 'user', 'url': 'no', 'category': 'no', 'rating': 'no', 'rating_count': 'no', 'ingredients': 'no', 'total': 'a', 'servings': 'no', 'yield': 'no', 'calories': 'b', 'carbohydrates_g': 'c', 'fat_g': 'd','protein_g': 'e','ingredients_parsed': 'no'}
recipes = recipes.append(user_df, ignore_index = True)
recipes.tail(2)

/tmp/ipykernel_23514/2606021746.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recipes = recipes.append(user_df, ignore_index = True)


,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed,recipes_id
19223,Easy Baked Zucchini Chips,https://www.allrecipes.com/recipe/242066/easy-...,appetizers-and-snacks,3.68,28,"['1 zucchini, thinly sliced ', ' 1 tablespoon...",30,2,2 servings,69.1,2.0,6.9,0.7,['flavored'],19223.0
19224,user,no,no,no,no,no,a,no,no,b,c,d,e,no,NaN


In [39]:
##How can I take user input and add it this function?
# def user_recommender(a, b, c, d, e):
    # total=float(a)
    # calories=float(b)
    # carbs=float(c)
    # fat=float(d)
    # protein=float(e)
    # user_input=[a,b,c,d,e]
    # normalize([user_input])
    # print([user_input])

In [40]:
# user_recommender(15, 400, 50, 9, 10)

In [41]:

##append it to the dataframe
# to_append = [5, 6]
# a_series = pd.Series(to_append, index = df.columns)
# df = df.append(a_series, ignore_index=True)

In [42]:
###Gigantic function, I hope it works 
def personalized_recommender(a, b, c, d, e):
    

IndentationError: expected an indented block (4283453877.py, line 3)